# Waterberry Farms: a realistic benchmark for multi-robot informative path planning

This notebook contains the code for generating the pictures and graphs for the paper. 

It illustrates various ways of running a __one day__ experiment on a __static environment__ with a __single robot__. The experiments compare estimator/path planner pairs. The path planners are simple static examples (lawnmower, spiral, random waypoint) and they are not intended to be state of the art. 

In [1]:
# allow imports from the main source directory
import sys
sys.path.append('..')
import logging
logging.basicConfig(level=logging.WARNING)

import pathlib

import pickle
import gzip as compress
import copy
import matplotlib.pyplot as plt
import matplotlib.lines as lines
import matplotlib.patches as patches
# configuring the fonts such that no Type 3 fonts are used
# requirement for ICRA
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams["figure.autolayout"] = True

import numpy as np
import timeit
import warnings

from InformationModel import StoredObservationIM
from WaterberryFarm import create_wbfe, WaterberryFarm, MiniberryFarm, WBF_IM_DiskEstimator, WBF_IM_GaussianProcess, WBF_Score_WeightedAsymmetric, get_datadir
from WbfExperiment import action_run_oneday, action_precompute_environment, get_geometry
from WbfFigures import graph_env_im, graph_scores, hook_create_pictures, end_of_day_graphs
from Policy import FollowPathPolicy, RandomWaypointPolicy
from PathGenerators import find_fixed_budget_spiral, generate_lawnmower, find_fixed_budget_lawnmower, generate_spiral_path, find_fixed_budget_spiral

logging.getLogger().setLevel(logging.WARNING)

# all the experiments results etc. should go into this directory
benchmark_dir = pathlib.Path(get_datadir(), "Benchmark-Paper")
benchmark_dir.mkdir(exist_ok=True)

# Precompute environments
* Precompute the evaluation of the environments. This allows the following simulations to run faster and to use the same environment for different path planning algorithms. 
* The precomputed data is saved into the corresponding directory under ```__Temporary/2024-03-07-WaterberryFarms_data```. Substitute for ```2024-03-07-WaterberryFarms``` whatever is the name of the directory above the github one. 

__Delete that directory if you want to run the simulation with differently parameterized environments.__

* This precalculates 50 days of evolution for all four environments. This is sufficient for the standard benchmark settings. 
* As this simulation is expensive for the large environments, be patient, it takes about __30 minutes__ on a fast machine if Waterberry is part of the environments. 


In [3]:
precompute = 50
# typenames = ["Miniberry-10", "Miniberry-30", "Miniberry-100", "Waterberry"]
typenames = ["Miniberry-10","Miniberry-30", "Miniberry-100"]
time_start_environment = 6

# typenames = ["Waterberry"]
# typenames = ["Miniberry-100"]
for typename in typenames:
    action_precompute_environment({"geometry": typename, "precompute-time": precompute})

# Generate pictures of the geometry. 
Generates pictures for the geometry of the various benchmark configurations (Miniberry-10,30,100 and Waterberry), and saves them into files $data/geometry_Miniberry-10.pdf etc.

FIXME: This could have been implemented with action load-environment

In [6]:
timepoints = [1, 3, 5, 10]
# typename = "Miniberry-10"

typenames_1 = ["Miniberry-10", "Miniberry-30", "Miniberry-100", "Waterberry"]
for typename in typenames_1:
    _, wbfe, _ = create_wbfe(saved = True, wbf_prec = None, typename = typename)
    geom = wbfe.geometry
    fig, ax = plt.subplots(1, figsize=(5,4))
    wbfe.geometry.visualize(ax)
    fig.tight_layout()
    plt.savefig(pathlib.Path(benchmark_dir, f"geometry_{typename}.pdf"))
    plt.savefig(pathlib.Path(benchmark_dir, f"geometry_{typename}.jpg"))

## Compare movement policies with the same estimator
Run one day experiments for comparing different policies with the same estimator. Visualize the results for each policy.

In [6]:
default_choices = {"geometry": "Miniberry-30", "action": "run", "scenario": "one-day-single-value", "visualize": 0,  "results-basedir": benchmark_dir}
default_choices["velocity"] = 1
default_choices["timesteps-per-day-override"] = 500 # 1000
default_choices["time-start-environment"] = 6
default_choices["hook-after-day"] = hook_create_pictures
 
# if force_run is false, we do not re-run an already run policy 
force_run = False
policies = []

# a random waypoint policy
geo = get_geometry(default_choices["geometry"])
policy = RandomWaypointPolicy(vel = 1, low_point = [geo["xmin"], geo["ymin"]], high_point = [geo["xmax"], geo["ymax"]], seed = 0)  
policy.name = "RandomWaypoint-Seed0"
policies.append(policy)

 # a lawnmower policy that covers the target area in one day uniformly
geo = get_geometry(default_choices["geometry"])
path = find_fixed_budget_lawnmower([0,0], geo["xmin"], geo["xmax"], geo["ymin"], geo["ymax"], geo["velocity"], time = geo["timesteps-per-day"])
policy = FollowPathPolicy(vel = geo["velocity"], waypoints = path, repeat = True)
policy.name = "FixedBudgetLawnmower"
policies.append(policy)

# a lawnmower policy that covers the tomatoes more densely than the strawberries
geo = get_geometry(default_choices["geometry"])
height = geo["ymax"] - geo["ymin"]
time = float(geo["timesteps-per-day"])
## FIXME: this has to search for a plan that can be executed in the time
time_safety = time / 10.0
time_strawberry = time / 3.0 - time_safety
time_tomato = time - time_strawberry - time_safety
path1 = find_fixed_budget_lawnmower([0, 0], geo["xmin"], geo["xmax"], geo["ymin"] + height//2, geo["ymin"] + height, geo["velocity"], time_tomato)
path2 = find_fixed_budget_lawnmower(path1[-1], geo["xmin"], geo["xmax"], geo["ymin"], geo["ymin"] + height // 2, geo["velocity"], time_strawberry)
# second path is supposed to start from the last of first
path = np.append(path1, path2, axis=0)           
policy = FollowPathPolicy(vel = geo["velocity"], waypoints = path, repeat = True)            
policy.name = "AdaptiveLawnMower"
policies.append(policy)

# a spiral policy 
geo = get_geometry(default_choices["geometry"])
path = find_fixed_budget_spiral([0,0], x_min = geo["xmin"], x_max = geo["xmax"], y_min = geo["ymin"], y_max = geo["ymax"], velocity = geo["velocity"], time = geo["timesteps-per-day"])
policy = FollowPathPolicy(vel = geo["velocity"], waypoints = path, repeat = True) 
policy.name = "Spiral"    
policies.append(policy)

picture_paths = []

for policy in policies:
    # do a dryrun, to get the path to the cached values
    choices = copy.copy(default_choices)
    choices["policy-code"] = policy
    choices["policy-name"] = policy.name
    choices["dryrun"] = True
    choices["estimator-name"] = "AD"
    choices["estimator-code"] = WBF_IM_DiskEstimator(geo["width"], geo["height"])
    choices["score-code"] = WBF_Score_WeightedAsymmetric()

    results = action_run_oneday(choices)
    path = results["results-path"]
    print(path)
    print(results.keys())
    # avoid re-running experiments
    if not path.exists() or force_run:
        choices["dryrun"] = False
        results = action_run_oneday(choices)
    else:
        with compress.open(path, "rb") as f:
            results = pickle.load(f)
    if "picture-path" in results:
        print("Picture path in results!")
        picture_paths.append(results["picture-path"])
    end_of_day_graphs(results, f"daily-summary-{results['policy-name']}-{results['estimator-name']}.pdf", plot_uncertainty = False)

/home/lboloni/Documents/Hackingwork/__Temporary/2022-01-15-MREM_data/Benchmark-Paper/res_Miniberry-30_1M_RandomWaypoint-Seed0
dict_keys(['geometry', 'action', 'scenario', 'visualize', 'results-basedir', 'velocity', 'timesteps-per-day-override', 'time-start-environment', 'hook-after-day', 'policy-code', 'policy-name', 'dryrun', 'estimator-name', 'estimator-code', 'score-code', 'im_resolution', 'typename', 'wbf', 'wbfe', 'savedir', 'exp-name', 'days', 'xmin', 'xmax', 'ymin', 'ymax', 'width', 'height', 'timesteps-per-day', 'robot', 'results-path'])


## Compare estimators in experiments
Run experiments to compare the estimators - the adaptive disk (AD) vs the Gaussian Process (GP). The Gaussian process is much slower, so it will take a quite longer time.

In [ ]:


default_choices = {"geometry": "Miniberry-30", "action": "run", "scenario": "one-day-single-value", "visualize": 0,  "results-basedir": benchmark_dir}
default_choices["velocity-override"] = 1
default_choices["timesteps-per-day-override"] = 500
# 500 # will be 500, was 1000
default_choices["time-start-environment"] = 6
default_choices["hook-after-day"] = hook_create_pictures

# cleaning up the warnings from the GP estimator
# logging.getLogger().setLevel(logging.ERROR)
warnings.simplefilter("ignore")
#warnings.filterwarnings(warnings.Warning, "ignore")

# if force_run is false, we do not re-run an already run policy 
force_run = False
for estimator in ["AD", "GP", "DE-10"]:
# for estimator in ["GP"]:
    # do a dryrun, to get the path to the cached values
    choices = copy.copy(default_choices)
    choices["dryrun"] = True

    geo = get_geometry(default_choices["geometry"])
    policy = RandomWaypointPolicy(vel = 1, low_point = [geo["xmin"], geo["ymin"]], high_point = [geo["xmax"], geo["ymax"]], seed = 0)  
    policy.name = "RandomWaypoint-Seed0"
    choices["policy-code"] = policy
    choices["policy-name"] = f"{policy.name}-{estimator}"
    choices["score-code"] = WBF_Score_WeightedAsymmetric()

    if estimator == "AD":
        choices["estimator-name"]  = "AD"
        choices["estimator-code"] = WBF_IM_DiskEstimator(geo["width"], geo["height"])
    elif estimator == "GP":
        choices["estimator-name"]  = "GP"
        choices["estimator-code"] = WBF_IM_GaussianProcess(geo["width"], geo["height"])        
    elif estimator == "DE-10":
        choices["estimator-name"]  = "DE-10"
        choices["estimator-code"] = WBF_IM_DiskEstimator(geo["width"], geo["height"], disk_radius=10)        

    results = action_run_oneday(choices)
    path = results["results-path"]
    print(path)
    # avoid re-running experiments
    if not path.exists() or force_run:
        choices["dryrun"] = False
        results = action_run_oneday(choices)
    else:
        with compress.open(path, "rb") as f:
            results = pickle.load(f)
    end_of_day_graphs(results, f"daily-summary-{results['policy-name']}.pdf", plot_uncertainty = False)

## Compare the performance of various combinations of path planner and estimator

In [ ]:
# gather all the results. As I am reading them in, generate the graphs.
allresults = []
for a in benchmark_dir.iterdir():
    if a.name.startswith("res_"):
        print(a)
        with compress.open(a, "rb") as f:
            results = pickle.load(f)
            allresults.append(results)

fig, ax_scores = plt.subplots(1, figsize=(6,4))
for results in allresults:
    ax_scores.plot(results["scores"], label = f'{results["policy-name"][len("benchmarkpaper-"):]}+{results["estimator-name"]}')
ax_scores.set_ylim(top=0)
ax_scores.set_xlabel("Time")
ax_scores.set_ylabel("Score")
ax_scores.set_title("Scores")
ax_scores.legend()
fig.tight_layout()


## Compare the results of different score functions
Verifying that different scoring functions are called

In [ ]:
default_choices = {"geometry": "Miniberry-30", "action": "run", "scenario": "one-day-single-value", "visualize": 0,  "results-basedir": benchmark_dir}
default_choices["velocity"] = 1
default_choices["timesteps-per-day-override"] = 500 # 1000
default_choices["time-start-environment"] = 6

# a random waypoint policy
geo = get_geometry(default_choices["geometry"])
policy = RandomWaypointPolicy(vel = 1, low_point = [geo["xmin"], geo["ymin"]], high_point = [geo["xmax"], geo["ymax"]], seed = 0)  

# create a set of score functions
scores = {}
scores["default"] = WBF_Score_WeightedAsymmetric()
scores["only-strawberry"] = WBF_Score_WeightedAsymmetric(strawberry_importance=1.0, strawberry_negative_importance=10.0, tomato_importance=0.0, tomato_negative_importance=10.0, soil_importance=0.0)
scores["only-tomato"] = WBF_Score_WeightedAsymmetric(strawberry_importance=0.0, strawberry_negative_importance=10.0, tomato_importance=1.0, tomato_negative_importance=10.0, soil_importance=0.0)
scores["only-soil"] = WBF_Score_WeightedAsymmetric(strawberry_importance=0.0, strawberry_negative_importance=10.0, tomato_importance=0.0, tomato_negative_importance=10.0, soil_importance=1.0)

for scorename, scorecode in scores.items():
    # do a dryrun, to get the path to the cached values
    choices = copy.copy(default_choices)
    choices["policy-code"] = policy
    choices["policy-name"] = f"random-{scorename}"
    choices["estimator-name"] = "AD"
    choices["estimator-code"] = WBF_IM_DiskEstimator(geo["width"], geo["height"])
    choices["score-code"] = scorecode

    choices["dryrun"] = True
    results = action_run_oneday(choices)
    path = results["results-path"]
    print(path)
    # avoid re-running experiments
    if not path.exists() or force_run:
        choices["dryrun"] = False
        results = action_run_oneday(choices)
    else:
        with compress.open(path, "rb") as f:
            results = pickle.load(f)
    end_of_day_graphs(results, f"daily-summary-{results['policy-name']}-{results['estimator-name']}.pdf", plot_uncertainty = False)

# Comparing the computational cost of different estimators
This code calculates the computational cost for different estimators (current GP and adaptive disk). This code takes about 10 min to run. 

In [ ]:
from logging import getLogger

logging.getLogger().setLevel(logging.INFO)
warnings.simplefilter("ignore")

typenames = ["Miniberry-10", "Miniberry-30", "Miniberry-100", "Waterberry"]

file = pathlib.Path(benchmark_dir, "timecomp")
totaltime = 500 # the time we extend if 
timestep = 10 # at what timesteps we do a measurement
stop_at = 10.0 # if a single estimate takes longer than this, we stop

if file.exists():
    with open(file, "rb") as f:
        res = pickle.load(f)
else:
    res = {}
    for estimator in ["AD", "GP"]:
        for typename in typenames:
            # break out, as GP Waterberry 10 is 160 seconds
            if estimator == "GP" and typename == "Waterberry":
                break
            wbf, wbfe, _ = create_wbfe(saved = True, wbf_prec = None, typename = typename)
            wbfe.proceed(time_start_environment)
            if estimator == "GP":
                wbfim = WBF_IM_DiskEstimator(wbf.width, wbf.height)
            else: 
                wbfim = WBF_IM_GaussianProcess(wbf.width, wbf.height)
            ## create some observations and add them...
            times = []
            ts = []
            t = 0
            while t < totaltime:
                for obs in range(timestep):
                    t = t + 1
                    x = np.random.random_integers(wbf.width-1)
                    y = np.random.random_integers(wbf.height-1)
                    position = [x, y, t] # fixme random position
                    obs = wbfe.get_observation(position)
                    wbfim.add_observation(obs)
                ## measure the time it takes here
                time = timeit.timeit("wbfim.proceed(1)", number=1,  globals=globals())
                getLogger().info(f"{t} = {time} {estimator} {typename}")
                if time > stop_at:
                    break
                times.append(time)
                ts.append(t)
            res[f"{typename}_{estimator}"] = [ts, times]
    with open(file, "wb") as f:
        pickle.dump(res, f)

In [ ]:

fig, ax = plt.subplots(1, 1, figsize=(6,4))

for key in res:
    ax.plot(res[key][0], res[key][1], label = key)
#ax.set_ylim(top=0)
ax.set_xlim(right=700)
ax.set_xlabel("Observations")
ax.set_ylabel("Time (seconds)")
#ax_scores.set_title("Scores")
ax.legend()
fig.tight_layout()
plt.savefig(pathlib.Path(benchmark_dir, "estimator_computational_cost.pdf"))


# Evolution of the environments at a list of specific timepoints

Generates figures for the state of the environment components at various timepoints. The default timepoints are the 1, 5, 10, 15 that are good for visualizing what goes on in the multiday experiments. 

The results are saved into $data/env-dynamics-{typename}-{value} files.

In [ ]:
typenames = ["Miniberry-10", "Miniberry-30", "Miniberry-100", "Waterberry"]
#typenames = ["Miniberry-100"]
values = ["tylcv", "ccr", "soil"]
timepoints = [1, 5, 10, 15]

for value in values:
    for typename in typenames:
        _, wbfe, _ = create_wbfe(saved = True, wbf_prec = None, typename = typename)
        fig, axs = plt.subplots(1, len(timepoints), figsize = (2.5 * len(timepoints), 2.5))
        #fig.suptitle(f"{typename} / {value}")
        ntt = 0
        for t in range(timepoints[-1]+1):
            if t >= timepoints[ntt]:
                if value == "tylcv":
                    val = wbfe.tylcv.value
                elif value == "ccr":
                    val = wbfe.ccr.value
                elif value == "soil":
                    val = wbfe.soil.value
                axs[ntt].imshow(val.T, vmin=0, vmax=1, cmap="gray", origin="lower")
                axs[ntt].set_title(f"t={timepoints[ntt]}")
                ntt = ntt + 1
            wbfe.proceed()
        fig.tight_layout()
        plt.savefig(pathlib.Path(benchmark_dir, f"env-dynamics-{typename}-{value}.pdf"))
        plt.savefig(pathlib.Path(benchmark_dir, f"env-dynamics-{typename}-{value}.jpg"))

